In [1]:
#共計有多少Twitter 1
import pyes
import elasticsearch
import json

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="text", value="cve")

ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

tagg1 = pyes.aggs.TermsAgg('user_id', field='uid')

qsearch = pyes.query.Search(bq)
qsearch.agg.add(tagg1)

rs = conn.search(query=qsearch , indices="twitter2" , type="tweet") 

print len(rs.aggs['user_id']['buckets'])
#print json.dumps(rs.aggs,indent=2)

47


In [3]:
#共計有多少Tweet 2
import pyes
import elasticsearch
import json

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="text", value="cve")

ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

rs = conn.search(query=bq , indices="twitter2" , type="tweet") 
print len(rs)
#print rs

265


每個月參與的Twitter數量長條圖

In [5]:
#每個月參與的Twitter數量長條圖 3
import pyes
import elasticsearch
import json

from bokeh.charts.attributes import CatAttr #
from bokeh.charts import Bar, show
from bokeh.io import push_notebook, show, output_notebook    , output_file
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="text", value="cve")

ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

DHagg = pyes.aggs.DateHistogramAgg('month', field='created_at', interval='month', sub_aggs=[])
tagg1 = pyes.aggs.TermsAgg('user_id', field='uid')
DHagg.sub_aggs.append(tagg1)

qsearch = pyes.query.Search(bq)
qsearch.agg.add(DHagg)
rs = conn.search(query=qsearch , indices="twitter2" , doc_types="tweet") 

twitter = [ len(rs.aggs['month']['buckets'][i]['user_id']['buckets']) for i in range(12)]

data = dict(value=twitter,
           month = ['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

#print data
####figure

p = Bar(data, color='navy', values='value', label=CatAttr(columns=['month'], sort=False), agg='sum', 
        xlabel='Month', ylabel='twitter量', title='每個月參與的Twitter數量長條圖(cve)')
show(p)

Loading BokehJS ...

每個月參與的Tweets數量長條圖

In [6]:
#每個月參與的Tweets數量長條圖 4
import pyes
import elasticsearch
import json

from bokeh.charts.attributes import CatAttr #
from bokeh.charts import Bar, show
from bokeh.io import push_notebook, show, output_notebook    , output_file
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="text", value="cve")

ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

DHagg = pyes.aggs.DateHistogramAgg('month', field='created_at', interval='month', sub_aggs=[])

qsearch = pyes.query.Search(bq)
qsearch.agg.add(DHagg)

rs = conn.search(query=qsearch , indices="twitter2" , doc_types="tweet") 

tweet = [rs.aggs['month']['buckets'][i]["doc_count"] for i in range(12)]

data = dict(value=tweet,
            month = ['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
#print data

####figure
p = Bar(data, color='navy', values='value', label=CatAttr(columns=['month'], sort=False), 
        xlabel='Month', ylabel='tweet量', title='每個月參與的Tweets數量長條圖(cve)')
show(p)

Loading BokehJS ...

箱型圖

In [7]:
import pyes
import elasticsearch
import json
import pandas as pd

from bokeh.charts import BoxPlot, output_file, show
from bokeh.charts.attributes import CatAttr #

from bokeh.io import push_notebook, show, output_notebook    , output_file
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="text", value="cve")

ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

DHagg = pyes.aggs.DateHistogramAgg('month', field='created_at', interval='month', sub_aggs=[])
tagg1 = pyes.aggs.TermsAgg('user_id', field='uid')
DHagg.sub_aggs.append(tagg1)

qsearch = pyes.query.Search(bq)
qsearch.agg.add(DHagg)
rs = conn.search(query=qsearch , indices="twitter2" , doc_types="tweet") 

twitter = [ len(rs.aggs['month']['buckets'][i]['user_id']['buckets']) for i in range(12)]
month = ['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

#
da = pd.DataFrame(columns=['month','twitter', 'tweet' ])

for j in range(12):
    for i in range(twitter[j]):
        
        da.loc[len(da)] = [month[j],rs.aggs['month']['buckets'][j]['user_id']['buckets'][i]["key"],
                     rs.aggs['month']['buckets'][j]['user_id']['buckets'][i]["doc_count"]]
#

####figure
p = BoxPlot(da, values='tweet', label=CatAttr(columns=['month'], sort=False)
            , color='navy', title='每個月各Twitter所提到CVE的箱型圖(cve)')
show(p)

#print da

Loading BokehJS ...

圓餅圖

In [8]:
import pyes
import elasticsearch
import json

from bokeh.charts import Donut, show
import pandas as pd

from bokeh.io import push_notebook, show, output_notebook, output_file
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

es_address='192.168.99.1:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 

tq = pyes.query.TermQuery(field="text", value="cve")
ESR = pyes.ESRange(field="created_at", from_value="Wed Jan 01 00:00:00 +0000 2014",
                   to_value="Wed Dec 31 23:59:59 +0000 2014", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)
rs = conn.search(query=bq , indices="twitter2" , doc_types="tweet") 
#
url_n=0
url_y=0
for i in range(len(rs)):
    rs_url = len(rs[i]["entities"]['urls'])
    if rs_url==0:
        url_n=1+url_n
    if rs_url!=0:
        url_y=1+url_y
#
#print url_y
#print url_n
#print len(rs)
####figure
data = {'name':['with URL','none URL'], 'number':[url_y,url_n]}
da = pd.DataFrame(data = data)
d= Donut(da, label='name', values='number', title='有URL與無URL的Tweets的比例圓餅圖(cve)')
show(d)

Loading BokehJS ...